In [ ]:
import numpy as np
import pandas as pd

In [ ]:
movies = pd.read_csv('tmdb_5000_movies.csv/tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv/tmdb_5000_credits.csv')

In [ ]:
movies.head()


In [ ]:
credits['crew'].head(1).values


In [ ]:
movies.isna().sum()
movies.shape
# movies.size
# movies.sample(5)
# movies.loc[movies['vote_average'].min()]
# credits.size

In [ ]:
movies = movies.merge(credits, on = 'title')

In [ ]:
# Find titles that appear more than once in either DataFrame
# movies['title'].value_counts()[movies['title'].value_counts() > 1]

# credits['title'].value_counts()[credits['title'].value_counts() > 1]
movies.info()

In [ ]:

#now here we seleting those columns that are getting be used in our project and we'll
# remove those those are not gonna play role here
# # *********Selected********
# genres
# keywords
# movie_id
# overview
# title
# cast 
# crew

In [ ]:
movies = movies[['movie_id','genres', 'title','keywords','overview','cast','crew']]

In [ ]:

movies.shape

In [ ]:
movies.isna().sum()

In [ ]:
movies.dropna(inplace = True)

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres

In [ ]:
import ast

def convert (obj):
    L = []
    for i in ast.literal_eval(obj):
     L.append(i['name'])
    return L
    

In [ ]:
# convert('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

In [ ]:
movies['genres'] = movies['genres'].apply(convert)

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
import ast
def convert2(obj):
    counter = 0
    L = []
    for i in ast.literal_eval(obj):
     if counter != 3 :
      L.append(i['name'])
      counter += 1
     else: break
        
    return L
     

 
movies['cast'] = movies['cast'].apply(convert2)

In [ ]:
movies.head()

In [ ]:
movies['crew'].head(1).values

In [ ]:
import ast
def convert3(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
             L.append(i['name'])
             break
    return L
     

In [ ]:
movies['crew'] = movies['crew'].apply(convert3)

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
movies.head()

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x :[i.replace(' ', '') for i in x])
movies['cast'] = movies['cast'].apply(lambda x :[i.replace(' ', '') for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x :[i.replace(' ', '') for i in x])
movies['overview'] = movies['overview'].apply(lambda x :[i.replace(' ', '') for i in x])


In [ ]:

movies['tags'] = movies['overview'] +movies['keywords'] +movies['genres'] +movies['cast'] + movies['crew']
movies['tags'][1]

In [ ]:
new_df = movies[['movie_id', 'title','tags']]

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x : " ".join(x))

In [ ]:
new_df['tags']= new_df['tags'].apply(lambda x : x.lower())

In [ ]:
new_df['title'][1234] 

In [ ]:

# import sys
# !{sys.executable} -m pip install scikit-learn


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=5000, stop_words='english')

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vectors

In [ ]:
# cv.get_feature_names_out()
features = cv.get_feature_names_out()
list(features)

In [ ]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:

def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return ' '.join(y)
    

In [ ]:
# stem('loving')
new_df['tags'] = new_df['tags'].apply(stem)

In [ ]:
new_df['tags']

In [ ]:

featuress = cv.get_feature_names_out()
list(featuress)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
 # print(sorted(list(enumerate(similarity[1])), reverse = True,key= lambda x:x[1]))
sorted_list = sorted(
    [(i, float(v)) for i, v in enumerate(similarity[0])],
    key=lambda x: x[1],
    reverse=True
)[1:6]
sorted_list



In [ ]:
# new_df.iloc[2409]

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(
    [(i, float(v)) for i, v in enumerate(distances)],
    key=lambda x: x[1],
    reverse=True)[1:6]
    for i in movie_list:
     print(new_df.iloc[i[0]].title)    


In [ ]:
recommend('The Martian') #WE have to Provide the movie name for recommendations here 